In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Hello every-one , I am sharing the basic approach to tackle this competition. This code is for beginners who are new to kaggle , pytorch and data-science. I have tried to explain each and every code cell in a layman terms so that many beginners who get confused to understand the code of high scoring notebooks can easily get what I have done here.
**This notebook scored 18.50 on public leaderboad(pb) and 17.87 during validation**

# Basic Imports

In [ ]:
import math #for mathematical operations
import time 
import os
from skimage import io, transform #for augmentation purposes
import PIL #for image processing
workers=2
working_dir = './'#for loading saved files to this location , will be used at the end

# CONFIG

No need to get confused, I am just using names instead of copying the link of the directory

In [ ]:
main_dir = '../input/petfinder-pawpularity-score' 
batch_size = 32
np.random.seed(100)

In [ ]:
import matplotlib.pyplot as plt # used for plotting
import torch #pytorch
from torchvision import datasets,transforms, models 
from torch import nn, optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts #scheduler used for finding global minima
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler

# Data preprocessing

In [ ]:
train_df= pd.read_csv(f'{main_dir}/train.csv')  
train_df.head(4)

In [ ]:
train_df.info()

In [ ]:
plt.hist(train_df.iloc[:,13],bins=50,facecolor ='r',alpha= 0.5)
plt.xlabel('pawpularity_score')
plt.ylabel('frequency')
plt.title('PAWPULARITY DISTRIBUTION')
plt.grid(True)

* Now comes the tricky part( not that tricky though!!). I have used basic OOPs , this way code looks clean and its easy to track the info.
* The code below , gives outthe pawpularity score , transformed image and annotations. This class so formed will be called in further code cells... and it consits of 3 methods __init__, __len__,__getitem__
* Basically this compitition deals with images and tabular data, so we have to merge both type of data .... in __get_item__ method, the path of the images have been added into the table of annotations according to image ids. In this manner Pawpularity score act as a label/ answers(supervised learning) for the image.



In [ ]:
class Pawpularity_Data(Dataset):
    def __init__(self,csv_file,img_dir,transform = transforms.ToTensor()):
        self.annotations_csv = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform =transform
        
    def __len__(self):
        return len(self.annotations_csv)
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.img_dir,self.annotations_csv.iloc[idx,0])
        image = PIL.Image.open(img_name + '.jpg')
        # Columns 1 to 12 contain the annotations
        annotations = np.array(self.annotations_csv.iloc[idx, 1:13])
        annotations = annotations.astype('float')
        # Column 13 has the scores
        score = np.array(self.annotations_csv.iloc[idx, 13])
        score = torch.tensor(score.astype('float')).view(1).to(torch.float32)
        # Apply the transforms
        image = self.transform(image)
        sample = [image, annotations, score]
        return sample

# Image Augmentations

* Image augmentation is done to deliberately increase the size of the dataset so that model gets trained on a large dataset, this makes models's prediction more robust
* Augmentations can be of many types but according to my testings..... this for this competition , flipping and rotation gave better results 
* Whereas augmentations like blurring, changing contrast and vertical flipping hampered the prediction

In [ ]:
# Test out the transforms on an image (images need to be made the same size for the dataset to work)
img_transforms = transforms.Compose([transforms.Resize(255),
                                     transforms.CenterCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.RandomRotation(20),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])])

img_transforms_valid = transforms.Compose([transforms.Resize(255),
                                           transforms.CenterCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                std=[0.229, 0.224, 0.225])])

Calling the class that we created earlier 

In [ ]:
train_data = Pawpularity_Data(f'{main_dir}/train.csv',f'{main_dir}/train',transform =img_transforms)
train_data.img_dir

In [ ]:
dataloader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)

In [ ]:
images, annotations, scores = next(iter(dataloader))
print(images.shape)
print(scores.shape)
print(annotations.shape)

# LOOKING AT IMAGES FROM DATA

In [ ]:
def de_norm(tensor):
    image = tensor.to('cpu').clone().detach()
    image = image.numpy().squeeze()
    image = image * np.array((0.229, 0.224, 0.225)).reshape(3, 1, 1) + np.array((0.485, 0.456, 0.406)).reshape(3, 1, 1)
    img = (image * 255).astype(np.uint8) # unnormalize
    return plt.imshow(np.transpose(img, (1, 2, 0)))

In [ ]:
im_numpy = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(20, 10))
# display 20 images
for idx in np.arange(8):
    ax = fig.add_subplot(2, 4, idx+1, xticks=[], yticks=[])
    de_norm(images[idx])
    ax.set_title(scores[idx].item())


# CNN MODEL ARCHITECTURE

BASIC IMPORTS

In [ ]:
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

In [ ]:
# Calculate the dense layer input size
# Padding of 1 and of 3 means no change in the image dimensions apart from pooling

sdim = 224/2/2/2/2/2 #maxpoolin layers reduce xy dimensions by 2
print(sdim)
print(sdim*sdim*256+12) # add in the annotations

# Important!!! dont skip this
Since this competition needs , kaggle internet to be switched off before submitting the predictions, common issue may arise when importing a pretrained model like..... this .... model = models.resnet50(pretrained=True)....this approach is not wrong but it will require internet to download imagenet weights... which will throw an error during submission since internet is off.

To deal with this .... open a separate notebook and load a model like mentioned above and then save the model in .pt file and download it and upload in your working notebook.

In [ ]:
model = torch.load('../input/resnet-pretrained/resnet_pretrained.pt')


# Classifier Head 
You can use any classifier but I have gone with ANN with 5 dense layers..

In [ ]:
# Disable gradients on all model parameters to freeze the weights
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(nn.Linear(2048, 1024),
                         nn.ReLU(),
                         nn.Linear(1024, 512),
                         nn.ReLU(),
                         nn.Linear(512, 256),
                         nn.ReLU(),
                         nn.Linear(256, 128),
                         nn.ReLU(),
                         nn.Linear(128, 1),
                         nn.Sigmoid())

for param in model.fc.parameters():
    param.requires_grad = True

In [ ]:
print(model)

In [ ]:
torch.manual_seed(40)

criterion = nn.MSELoss(reduction='sum')


#Adam with L2 regularization
optimizer = optim.AdamW(model.parameters(), lr=0.0008, weight_decay=0.2)

scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,10,
    T_mult=1,
    eta_min=0,
    last_epoch=-1,
    verbose=False,
)

In [ ]:
# Load a small batch to test out the forward pass

train_dataset = Pawpularity_Data(f'{main_dir}/train.csv', f'{main_dir}/train', transform=img_transforms)
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
images, annotations, scores = next(iter(dataloader))

In [ ]:
# Test out the forward pass on a single batch
# RMSE before any training (with random parameters): 
with torch.no_grad():
    train_loss = 0.0
    output = model(images)*100
    loss = criterion(output, scores)
    math.sqrt(loss.item()/64)

In [ ]:
print(scores.dtype)
print(output.dtype)
print(torch.mean(output))
print(torch.std(output))

In [ ]:
## Load and set up the final training and validation dataset (use different transforms)

train_data = Pawpularity_Data(f'{main_dir}/train.csv', f'{main_dir}/train', transform=img_transforms)
valid_data = Pawpularity_Data(f'{main_dir}/train.csv', f'{main_dir}/train', transform=img_transforms_valid)

np.random.seed(100)

# obtain random indices that will be used for traingin/validation split
valid_size = 0.2
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           sampler=train_sampler, num_workers=workers,
                                           pin_memory=True) 
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers=workers,
                                           pin_memory=True) 

print(len(train_loader)*batch_size)
print(len(valid_loader)*batch_size)

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
device = torch.cuda.get_device_name()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print(f'CUDA is available!  Training on GPU {device}...')

# TRAINING THE MODEL

In [ ]:
# number of epochs to train the model
# Use 40 epochs

if train_on_gpu:
    model.cuda()

n_epochs = 70

valid_loss_min = np.Inf # track change in validation loss

train_losses, valid_losses = [], []

for epoch in range(1, n_epochs+1):
    
    start = time.time()
    current_lr = scheduler.get_last_lr()[0]
    
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    # put in training mode (enable dropout)
    model.train()
    for images, annotations, scores in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            images, annotations, scores = images.cuda(), annotations.cuda(), scores.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        # the annotations get added in the dense layers
        output = model(images)*100
        # print(output.dtype)
        # print(scores.dtype)
        # calculate the batch loss
        loss = criterion(output, scores)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()
        
    ######################    
    # validate the model #
    ######################
    # eval mode (no dropout)
    model.eval()
    with torch.no_grad():
        for images, annotations, scores in valid_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                images, annotations, scores = images.cuda(), annotations.cuda(), scores.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(images)*100
            # calculate the batch loss
            loss = criterion(output, scores)
            # update average validation loss 
            valid_loss += loss.item()
    
    # calculate RMSE
    train_loss = math.sqrt(train_loss/len(train_loader.sampler))
    valid_loss = math.sqrt(valid_loss/len(valid_loader.sampler))
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
        
    # increment learning rate decay
    scheduler.step()
    
    # print training/validation statistics 
    # print(f'Epoch: {e}, {float(time.time() - start):.3f} seconds, lr={optimizer.lr}')
    print('Epoch: {}, time: {:.3f}s, lr: {:.6f} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, float(time.time() - start), current_lr, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), f'{working_dir}pawpularity_best_model.pt')
        valid_loss_min = valid_loss    

In [ ]:
# Load the best performing model on the validation set
model.load_state_dict(torch.load(f'{working_dir}pawpularity_best_model.pt'))

In [ ]:
class PawpularityTestDataset(Dataset):
    """Dataset connecting dog images to the score and annotations"""

    def __init__(self, csv_file, img_dir, transform=transforms.ToTensor()):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            img_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """

        self.annotations_csv = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations_csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.img_dir,
                                self.annotations_csv.iloc[idx, 0])

        # load each image in PIL format for compatibility with transforms
        image = PIL.Image.open(img_name + '.jpg')

        annotations = np.array(self.annotations_csv.iloc[idx, 1:13])
        annotations = annotations.astype('float')

        # Apply the transforms
        image = self.transform(image)

        sample = [image, annotations]
        return sample

In [ ]:
## Load the test dataset
img_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor()])

test_data = PawpularityTestDataset(f'{main_dir}/test.csv', f'{main_dir}/test', transform=img_transforms_valid)

batch_size = min(len(test_data), 32)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=workers) 

In [ ]:
test_df = pd.read_csv(f'{main_dir}/test.csv')
test_df.head(10)

In [ ]:
# Step through with a reasonable batch size and build up the output dataset

model.eval()
outputs = []
for images, annotations in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        images, annotations = images.cuda(), annotations.cuda()
    test_output = model(images)*100
    outputs.extend(list(test_output.cpu().detach().numpy().reshape(len(test_output),)))
    
img_names = list( test_df.iloc[:, 0].values)
outputs = [round(x, 2) for x in outputs]

output_df = pd.DataFrame({'Id': img_names, 'Pawpularity': outputs})
output_df.head(10)

In [ ]:
# Write the output in the required format
output_df.to_csv('submission.csv', index=False)